In [22]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

In [23]:
df = pd.read_csv('taxi.csv')
df.head(5)

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response
0,1105373,5,20,6080,174182,55.818842,37.334562,55.814567,37.355010,-1.000,-1.000,Economy,private,0
1,759733,5,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,Standard,private,1
2,416977,6,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,Economy,private,0
3,889660,2,6,6080,163522,55.745922,37.421748,55.743469,37.431130,-1.000,-1.000,Economy,private,1
4,1120055,4,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,Economy,private,1


In [24]:
df.info() #в исходном дата сете нет пропусков!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
offer_gk                  100000 non-null int64
weekday_key               100000 non-null int64
hour_key                  100000 non-null int64
driver_gk                 100000 non-null int64
order_gk                  100000 non-null int64
driver_latitude           100000 non-null float64
driver_longitude          100000 non-null float64
origin_order_latitude     100000 non-null float64
origin_order_longitude    100000 non-null float64
distance_km               100000 non-null float64
duration_min              100000 non-null float64
offer_class_group         100000 non-null object
ride_type_desc            100000 non-null object
driver_response           100000 non-null int64
dtypes: float64(6), int64(6), object(2)
memory usage: 10.7+ MB


In [25]:
df.describe()

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,driver_response
count,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,5.662189e+05,3.159860,13.827790,3446.376360,460867.01291,55.700824,37.492292,55.772440,37.541637,18.393518,23.134841,0.492480
std,3.259360e+05,2.054222,7.306496,2038.738677,266064.98953,2.135903,1.454640,0.665985,0.467758,41.129296,33.890105,0.499946
min,4.000000e+00,0.000000,0.000000,14.000000,18.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,2.855552e+05,1.000000,8.000000,1421.000000,229178.50000,55.708651,37.413497,55.708032,37.407117,-1.000000,-1.000000,0.000000
50%,5.656715e+05,3.000000,16.000000,3506.500000,462420.00000,55.772904,37.533664,55.772561,37.532537,10.411500,19.700000,0.000000
75%,8.487198e+05,5.000000,20.000000,5242.000000,690894.00000,55.960667,37.624113,55.962000,37.624358,30.665250,37.867000,1.000000
max,1.130355e+06,6.000000,23.000000,6806.000000,920344.00000,56.174097,38.280547,56.311121,38.243608,5234.283000,3958.283000,1.000000


наблюдения: 
1. примем вос = 0, пон = 1 итд
2. категириальные переменные offer_class_group и ride_type_desc преобразовать, уменьшив разменрость насколько это возможно. 
3. у признаков времени в пути и расстояния тоже есть значения -1. Тут нужно принять решение, в завасимости от количества -1. Либо это выбросы и мы удаляем все значения -1 или эти значения говорят о том что в такси был выключен счетчик во время поездки или другвя поломка, но поездка состоялась в итоге. Тогда я поменяю -1 на значения медианы


___________________

In [26]:
#рассмотрим категориальную переменную "класс"
df.offer_class_group.value_counts()

Economy     48335
Standard    47288
Premium      1773
Delivery     1290
Kids          588
XL            464
VIP           225
VIP+           29
Test            8
Name: offer_class_group, dtype: int64

идея: сократить размерность до 3. 0- Economy,Delivery; 1-Standard,Kids,XL,Test ; 2- Premium,VIP,VIP+

In [50]:
dict_class_to_num = {
    'Economy': 0,
    'Standard': 1,
    'Premium': 2,
    'Delivery': 0,
    'Kids': 1,
    'VIP': 2,
    'XL': 1,
    'VIP+': 2,
    'Test': 1,
}
df['offer_class_group'] = df['offer_class_group'].apply(lambda x: dict_class_to_num[x])

In [51]:
df['offer_class_group'].head()

0    0
1    1
2    0
3    0
4    0
Name: offer_class_group, dtype: int64

___________________

In [52]:
#рассмотрим категориальную переменную тип поездки
df.ride_type_desc.value_counts()

private      91207
business      8533
affiliate      224
SMB             36
Name: ride_type_desc, dtype: int64

идея: свести переменную к значениям 1 и 0. 0 - личная поездка 1- все остальное

In [61]:
ride_type= []
for i in df.ride_type_desc:
    if('private' in i):
        ride_type.append(0)
    else:
        ride_type.append(1)

In [62]:
df.ride_type_desc = ride_type

In [63]:
df.head(10)

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response,class_group
0,1105373,5,20,6080,174182,55.818842,37.334562,55.814567,37.355010,-1.000,-1.000,0,0,0,0
1,759733,5,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,1,0,1,1
2,416977,6,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,0,0,0,0
3,889660,2,6,6080,163522,55.745922,37.421748,55.743469,37.431130,-1.000,-1.000,0,0,1,0
4,1120055,4,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,0,0,1,0
5,1058203,6,15,6080,360328,55.818963,37.361033,55.814827,37.354074,6.747,9.800,0,0,0,0
6,103326,2,11,6080,615584,55.753508,37.663742,55.757251,37.659064,-1.000,-1.000,1,1,1,1
7,128861,5,20,6080,254374,55.839318,37.248862,55.839134,37.304101,1.517,3.933,0,0,0,0
8,493006,5,20,6080,753429,55.818748,37.338240,55.835764,37.294710,6.222,15.433,0,0,0,0
9,953983,6,7,6080,533820,55.839155,37.251947,55.833198,37.312197,10.410,23.083,0,0,0,0


___________________

In [64]:
df.distance_km.value_counts().head()

-1.000     26207
 0.000       152
 7.718        39
 56.447       35
 11.014       32
Name: distance_km, dtype: int64

In [65]:
# заменим значения -1 в столбце distance_km на значение медианы

In [66]:
#возьмем только те значения, которые больше -1.Чтобы найти верную медиану
km_known = df.distance_km.loc[df.distance_km > -1]

In [67]:
km_known.median()

18.607

In [68]:
df.distance_km = [ km_known.median() if x==-1 else x for x in df.distance_km]

In [69]:
# значения 0 пока оставим. Может быть это отмененнные поезки

In [70]:
df.distance_km.value_counts().head()

18.607    26209
0.000       152
7.718        39
56.447       35
11.014       32
Name: distance_km, dtype: int64

___________________

In [71]:
# заменим значения -1 в столбце duration_min на значение медианы

In [72]:
df.duration_min.value_counts().head()

-1.000     26207
 0.000       152
 21.467       65
 56.833       64
 23.333       56
Name: duration_min, dtype: int64

In [73]:
time_median = df.duration_min.loc[df.duration_min > -1].median()

In [74]:
time_median 

28.183000000000003

In [75]:
df.duration_min = [time_median if x ==-1 else x for x in df.duration_min]

In [76]:
df.duration_min.value_counts().head()

28.183    26253
0.000       152
21.467       65
56.833       64
23.333       56
Name: duration_min, dtype: int64

___________________

In [77]:
df.head()

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response,class_group
0,1105373,5,20,6080,174182,55.818842,37.334562,55.814567,37.355010,18.607,28.183,0,0,0,0
1,759733,5,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,1,0,1,1
2,416977,6,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,0,0,0,0
3,889660,2,6,6080,163522,55.745922,37.421748,55.743469,37.431130,18.607,28.183,0,0,1,0
4,1120055,4,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,0,0,1,0


___________________

 разберемся с днями недели. Логично предположить что в выходные и конец недели спрос на такси меняется, поэтому понизим размерность до 0 и 1. Где 0 - это будни, а 1 - выходные(пт,сб,вс). 
 выходные будут 5,6,0

In [80]:
holiday = [5,6,0]
df['weekday'] = [1 if x in holiday else 0 for x in df.weekday_key]
df = df.drop(['weekday_key'], axis = 1)

In [81]:
df.head()

,offer_gk,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response,class_group,weekday
0,1105373,20,6080,174182,55.818842,37.334562,55.814567,37.355010,18.607,28.183,0,0,0,0,1
1,759733,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,1,0,1,1,1
2,416977,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,0,0,0,0,1
3,889660,6,6080,163522,55.745922,37.421748,55.743469,37.431130,18.607,28.183,0,0,1,0,0
4,1120055,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,0,0,1,0,0


___________________

Преобразуем столбец hour_key. применим StandardScaler


In [82]:
Y = pd.Series(df.hour_key)

In [83]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df.hour = scaler.fit_transform(pd.DataFrame(Y))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


In [84]:
df.hour.std(axis = 0)# =1

array([1.])

In [85]:
np.mean(df.hour,axis = 0)# ~ 0

array([-4.94537744e-17])

In [86]:
df['hour_norm'] = df.hour

___________________

In [87]:
df.head()

,offer_gk,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response,class_group,weekday,hour_norm
0,1105373,20,6080,174182,55.818842,37.334562,55.814567,37.355010,18.607,28.183,0,0,0,0,1,0.844761
1,759733,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,1,0,1,1,1,0.023570
2,416977,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,0,0,0,0,1,0.023570
3,889660,6,6080,163522,55.745922,37.421748,55.743469,37.431130,18.607,28.183,0,0,1,0,0,-1.071352
4,1120055,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,0,0,1,0,0,0.297300


построим первую модель. Возьмем для тренировки атрибуты distance_km, duration_min,offer_class_group,ride_type_desc,hour,weekday
Используем классификатор "случайный лес"


In [88]:
df_1 = df[['distance_km','duration_min','offer_class_group','ride_type_desc','weekday','hour_norm']]
df_1.describe()

,distance_km,duration_min,offer_class_group,ride_type_desc,weekday,hour_norm
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05
mean,23.531924,30.782830,0.524020,0.087930,0.491610,-1.858935e-16
std,39.581052,30.725734,0.538485,0.283194,0.499932,1.000005e+00
min,0.000000,0.000000,0.000000,0.000000,0.000000,-1.892543e+00
25%,10.985000,20.400000,0.000000,0.000000,0.000000,-7.976216e-01
50%,18.607000,28.183000,1.000000,0.000000,0.000000,2.972999e-01
75%,30.665250,37.867000,1.000000,0.000000,1.000000,8.447607e-01
max,5234.283000,3958.283000,2.000000,1.000000,1.000000,1.255356e+00


In [89]:
X = df_1 
y = df['driver_response']
print(X.shape)
print(y.shape)

(100000, 6)
(100000,)


In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=30)

In [91]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss, r2_score
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                            random_state=0)
rf_model = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))
y_prob = rf_model.predict_proba(X_test)
print(log_loss(y_test, y_prob))

0.6404
0.6348307754121079


___________________

Предыдущую модель мы строили без учета координат водителя и заказчика. Вычислим расстояние между ними и включим этот аргумент в следующую модель

In [92]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [93]:
from geopy.distance import vincenty


df['driver_order_dis'] = df[['origin_order_latitude', 'origin_order_longitude', 'driver_latitude', 'driver_longitude']].apply(
    lambda rec: vincenty((rec['origin_order_latitude'], rec['origin_order_longitude']),(rec['driver_latitude'], rec['driver_longitude'])).km, axis = 1)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  """


In [114]:
df.head(20)

,offer_gk,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response,class_group,weekday,hour_norm,driver_order_dis
0,1105373,20,6080,174182,55.818842,37.334562,55.814567,37.355010,18.607,28.183,0,0,0,0,1,0.844761,1.367355
1,759733,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,1,0,1,1,1,0.023570,3.423260
2,416977,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,0,0,0,0,1,0.023570,0.411356
3,889660,6,6080,163522,55.745922,37.421748,55.743469,37.431130,18.607,28.183,0,0,1,0,0,-1.071352,0.649440
4,1120055,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,0,0,1,0,0,0.297300,1.064116
5,1058203,15,6080,360328,55.818963,37.361033,55.814827,37.354074,6.747,9.800,0,0,0,0,1,0.160435,0.634321
6,103326,11,6080,615584,55.753508,37.663742,55.757251,37.659064,18.607,28.183,1,1,1,1,0,-0.387026,0.509840
7,128861,20,6080,254374,55.839318,37.248862,55.839134,37.304101,1.517,3.933,0,0,0,0,1,0.844761,3.460869
8,493006,20,6080,753429,55.818748,37.338240,55.835764,37.294710,6.222,15.433,0,0,0,0,1,0.844761,3.321383
9,953983,7,6080,533820,55.839155,37.251947,55.833198,37.312197,10.410,23.083,0,0,0,0,1,-0.934487,3.832871


_____________________

Пыталась реализовать вычисления расстояния более привычным способом через создание функции, но не смотгла справится с ошибкой "cannot convert the series to <class 'float'>
"

In [103]:
from math import sin, cos, sqrt, atan2, radians,acos
def distance(lat1,lon1,lat2,lon2):
    lat1,lon1,lat2,lon2 = map(radians, [lat1,lon1,lat2,lon2])
    a = (6373* (acos(sin(lat1)*sin(lat2)+cos(lat1)*cos(lat2)*cos(lon1-lon2))))
    return float(a)

In [104]:
lat1 = df.origin_order_latitude
lon1 = df.origin_order_longitude
lat2 = df.driver_latitude
lon2 = df.driver_longitude

In [105]:
lat1.dtype

dtype('float64')

In [106]:
df_rec = distance(lat1,lon1,lat2,lon2)

TypeError: cannot convert the series to <class 'float'>

_____________________

In [95]:
df_2 = df[['distance_km',
          'duration_min',
          'offer_class_group',
          'ride_type_desc',
          'weekday',
          'hour_norm',
          'driver_order_dis']]

In [96]:
# строим модель использую классификатор "случайный лес"
X = df_2
y = df['driver_response']
print(X.shape)
print(y.shape)

(100000, 7)
(100000,)


In [97]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=30)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss, r2_score
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                            random_state=0)
rf_model = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))
y_prob = rf_model.predict_proba(X_test)
print(log_loss(y_test, y_prob))

0.6479
0.6279076790221692


Модель сработала почти аналогично, присутствуют небольшие улучшения.

_____________________

Преобразуем по другому атрибут времени заказа такси 

In [101]:
hours = df['hour_key']

In [106]:
hours.value_counts()

23    8184
21    7398
22    6881
0     6646
20    6566
19    6369
18    5889
17    5028
16    4500
15    4249
14    3584
8     3550
1     3467
13    3355
9     3298
12    3169
11    2890
10    2840
7     2482
2     2275
4     2193
5     1809
3     1768
6     1610
Name: hour_key, dtype: int64

Можно заметить что с 15 до полуночи заказов становится больше. разделим атрибут на 0 и 1, где 0 время с 1 до 14, а 1 с 15 до 0

In [111]:
hours = [0 if x in range(1,15) else 1 for x in hours]

In [115]:
df['hours_count'] = hours

In [120]:
df.head()

,offer_gk,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response,class_group,weekday,hour_norm,driver_order_dis,hours_count
0,1105373,20,6080,174182,55.818842,37.334562,55.814567,37.355010,18.607,28.183,0,0,0,0,1,0.844761,1.367355,0
1,759733,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,1,0,1,1,1,0.023570,3.423260,0
2,416977,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,0,0,0,0,1,0.023570,0.411356,0
3,889660,6,6080,163522,55.745922,37.421748,55.743469,37.431130,18.607,28.183,0,0,1,0,0,-1.071352,0.649440,1
4,1120055,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,0,0,1,0,0,0.297300,1.064116,0


_____________________

In [121]:
df_3 = df[['distance_km',
          'duration_min',
          'offer_class_group',
          'ride_type_desc',
          'weekday',
          'hours_count',
          'driver_order_dis']]

In [124]:
# строим модель использую классификатор "случайный лес"
X = df_3
y = df['driver_response']
print(X.shape)
print(y.shape)

(100000, 7)
(100000,)


In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=30)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss, r2_score
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                            random_state=0)
rf_model = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))
y_prob = rf_model.predict_proba(X_test)
print(log_loss(y_test, y_prob))

0.6365
0.6301702322280258


результаты модели незначительно ухудштлись